<a href="https://colab.research.google.com/github/dk-wei/dl-recomm-algo-implementation/blob/main/DeepFM_(FFM%E5%92%8CDNN%E7%9A%84%E7%89%B9%E5%BE%81%E7%BB%93%E5%90%88)_StepByStep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

代码来源：https://github.com/princewen/tensorflow_practice/blob/master/recommendation/Basic-DeepFM-model/DeepFM-StepByStep.ipynb

DeepFM 源代码：https://github.com/ChenglongChen/tensorflow-DeepFM    

讲解博客: https://www.jianshu.com/p/6f1c2643d31bhttps://www.jianshu.com/p/6f1c2643d31b

![](https://d2l.ai/_images/rec-deepfm.svg)

In [58]:
%tensorflow_version 1.x


数据来源：   

https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/data

Porto Seguro’s Safe Driver Prediction     
Predict if a driver will file an insurance claim next year.

Data Description:

In this competition, you will predict the probability that an auto insurance policy holder files a claim.

In the train and test data, features that belong to similar groupings are tagged as such in the feature names (e.g., `ind`, `reg`, `car`, `calc`). In addition, feature names include the postfix `bin` to indicate binary features and `cat` to indicate categorical features. Features without these designations are either continuous or ordinal. Values of `-1` indicate that the feature was missing from the observation. The `target` columns signifies whether or not a claim was filed for that policy holder.

数据EDA分析地址：https://blog.csdn.net/qq_37195507/article/details/78553581

In [59]:
import pandas as pd

TRAIN_FILE = "https://raw.githubusercontent.com/princewen/tensorflow_practice/master/recommendation/Basic-DeepFM-model/data/train.csv"
TEST_FILE = "https://raw.githubusercontent.com/princewen/tensorflow_practice/master/recommendation/Basic-DeepFM-model/data/test.csv"

NUMERIC_COLS = [
    "ps_reg_01", "ps_reg_02", "ps_reg_03",
    "ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"
]

IGNORE_COLS = [
    "id", "target",
    "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04",
    "ps_calc_05", "ps_calc_06", "ps_calc_07", "ps_calc_08",
    "ps_calc_09", "ps_calc_10", "ps_calc_11", "ps_calc_12",
    "ps_calc_13", "ps_calc_14",
    "ps_calc_15_bin", "ps_calc_16_bin", "ps_calc_17_bin",
    "ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"
]

dfTrain = pd.read_csv(TRAIN_FILE)
dfTest = pd.read_csv(TEST_FILE)
print(dfTrain.head(5))

   id  target  ps_ind_01  ...  ps_calc_18_bin  ps_calc_19_bin  ps_calc_20_bin
0   7       0          2  ...               0               0               1
1   9       0          1  ...               0               1               0
2  13       0          5  ...               0               1               0
3  16       0          0  ...               0               0               0
4  17       0          0  ...               1               1               0

[5 rows x 59 columns]


In [60]:
df = pd.concat([dfTrain,dfTest])
df.head(10)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0.0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0.0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0.0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0.0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0.0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
5,19,0.0,5,1,4,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,1.8,2.332649,10,0,-1,0,0,14,1,1,0,1,104,2,0.445982,0.879049,0.406202,3.000000,0.7,0.8,0.4,3,1,8,2,11,3,8,4,2,0,9,0,1,0,1,1,1
6,20,0.0,2,1,3,1,0,0,1,0,0,0,0,0,0,0,8,1,0,0,0.6,0.1,0.617454,6,1,-1,0,1,11,1,1,0,1,99,2,0.316228,0.639683,0.368782,3.162278,0.2,0.6,0.5,2,2,8,1,8,3,10,3,0,0,10,0,1,0,0,1,0
7,22,0.0,5,1,4,0,0,1,0,0,0,0,0,0,0,0,13,1,0,0,0.7,0.4,0.607248,11,1,0,0,0,11,1,1,2,1,30,3,0.446990,0.900574,0.374833,3.316625,0.1,0.5,0.1,1,2,7,1,6,1,13,7,1,3,6,1,0,1,0,1,0
8,26,0.0,5,1,3,1,0,0,0,1,0,0,0,0,0,0,6,1,0,0,0.9,0.7,0.901388,10,1,-1,0,1,14,1,1,0,1,68,3,0.400000,0.780641,0.407431,2.828427,0.9,0.8,0.6,3,1,7,3,9,4,11,4,2,1,5,0,1,0,0,0,1
9,28,1.0,1,1,2,0,0,0,1,0,0,0,0,0,0,0,4,0,0,1,0.9,1.4,2.316652,11,0,0,1,0,14,1,1,2,1,104,2,0.447214,1.458184,0.390256,3.605551,0.7,0.8,0.8,2,2,8,2,9,1,11,3,5,0,6,0,1,0,0,1,0


对于feature需要准备两个dict:
1. feature_index
2. feature_value

特征索引feature_index的作用是从embedding矩阵中抽取指定行的向量（查表提取），而特征取值feature_value则作为原始数据用来乘以取出的embedding行进行一个embedding的操作或者说是xi*vi的操作。 


下面为提取方法:    
![](https://pic4.zhimg.com/80/v2-05ad70142b9993720d3d6687f0587ee3_1440w.jpg)

参考博客：https://zhuanlan.zhihu.com/p/148836639

In [85]:
feature_dict = {} 
#特征位置（离散化后的特征位置）
total_feature = 0 
for col in df.columns:
     if col in IGNORE_COLS:#如果特征再忽略名单中
         continue
     elif col in NUMERIC_COLS:                 #如果特征是连续数值
         feature_dict[col] = total_feature     #直接存储一个特征位置，因为连续特征不做离散化，因此在独热特征中只占一个位置
         total_feature += 1                    #特征位置后移一个
     else:                                     #如果特征是离散特征
         unique_val = df[col].unique()         #得到该离散特征的所有特征取值
        #不同于连续型只存储一个位置，这里存放的是一个字典，字典中存储了该特征的每一个特征值以及特征值在独热特征中的位次
         feature_dict[col] = dict(zip(unique_val,range(total_feature,len(unique_val) + total_feature)))
         total_feature += len(unique_val)      #位次加上特征值的数量（因为这一个离散特征就要占用多个位次的独热特征） 

In [86]:
print(total_feature)
print(feature_dict)    # freq of elements in each col

254
{'ps_ind_01': {2: 0, 1: 1, 5: 2, 0: 3, 4: 4, 3: 5, 6: 6, 7: 7}, 'ps_ind_02_cat': {2: 8, 1: 9, 4: 10, 3: 11, -1: 12}, 'ps_ind_03': {5: 13, 7: 14, 9: 15, 2: 16, 0: 17, 4: 18, 3: 19, 1: 20, 11: 21, 6: 22, 8: 23, 10: 24}, 'ps_ind_04_cat': {1: 25, 0: 26, -1: 27}, 'ps_ind_05_cat': {0: 28, 1: 29, 4: 30, 3: 31, 6: 32, 5: 33, -1: 34, 2: 35}, 'ps_ind_06_bin': {0: 36, 1: 37}, 'ps_ind_07_bin': {1: 38, 0: 39}, 'ps_ind_08_bin': {0: 40, 1: 41}, 'ps_ind_09_bin': {0: 42, 1: 43}, 'ps_ind_10_bin': {0: 44, 1: 45}, 'ps_ind_11_bin': {0: 46, 1: 47}, 'ps_ind_12_bin': {0: 48, 1: 49}, 'ps_ind_13_bin': {0: 50, 1: 51}, 'ps_ind_14': {0: 52, 1: 53, 2: 54, 3: 55}, 'ps_ind_15': {11: 56, 3: 57, 12: 58, 8: 59, 9: 60, 6: 61, 13: 62, 4: 63, 10: 64, 5: 65, 7: 66, 2: 67, 0: 68, 1: 69}, 'ps_ind_16_bin': {0: 70, 1: 71}, 'ps_ind_17_bin': {1: 72, 0: 73}, 'ps_ind_18_bin': {0: 74, 1: 75}, 'ps_reg_01': 76, 'ps_reg_02': 77, 'ps_reg_03': 78, 'ps_car_01_cat': {10: 79, 11: 80, 7: 81, 6: 82, 9: 83, 5: 84, 4: 85, 8: 86, 3: 87, 0: 8

需要对数据根据FFM和FM的要求，进行格式转化

In [63]:
"""
对训练集进行转化
"""
print(dfTrain.columns)
train_y = dfTrain[['target']].values.tolist()
dfTrain.drop(['target','id'],axis=1,inplace=True)
train_feature_index = dfTrain.copy()
train_feature_value = dfTrain.copy()

for col in train_feature_index.columns:
    if col in IGNORE_COLS:
        train_feature_index.drop(col,axis=1,inplace=True)
        train_feature_value.drop(col,axis=1,inplace=True)
        continue
    elif col in NUMERIC_COLS:   
        train_feature_index[col] = feature_dict[col]
    else:
        train_feature_index[col] = train_feature_index[col].map(feature_dict[col])
        train_feature_value[col] = 1

Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],


In [64]:
train_feature_value.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.7,0.2,0.718070,1,1,1,1,1,1,1,1,1,1,1,1,0.400000,0.883679,0.370810,3.605551
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.8,0.4,0.766078,1,1,1,1,1,1,1,1,1,1,1,1,0.316228,0.618817,0.388716,2.449490
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0,-1.000000,1,1,1,1,1,1,1,1,1,1,1,1,0.316228,0.641586,0.347275,3.316625
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.9,0.2,0.580948,1,1,1,1,1,1,1,1,1,1,1,1,0.374166,0.542949,0.294958,2.000000
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.7,0.6,0.840759,1,1,1,1,1,1,1,1,1,1,1,1,0.316070,0.565832,0.365103,2.000000


In [65]:
train_feature_index.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,0,8,13,25,28,36,38,40,42,44,46,48,50,52,56,70,72,74,76,77,78,79,92,94,97,107,110,128,131,133,139,142,246,250,251,252,253
1,1,9,14,26,28,36,39,41,42,44,46,48,50,52,57,70,73,75,76,77,78,80,92,94,97,108,111,128,132,134,139,143,247,250,251,252,253
2,2,10,15,25,28,36,39,41,42,44,46,48,50,52,58,71,73,74,76,77,78,81,92,94,97,108,112,128,132,134,139,144,248,250,251,252,253
3,3,9,16,26,28,37,39,40,42,44,46,48,50,52,59,71,73,74,76,77,78,81,92,95,97,107,111,128,132,135,139,145,248,250,251,252,253
4,3,8,17,25,28,37,39,40,42,44,46,48,50,52,60,71,73,74,76,77,78,80,92,94,97,108,112,128,132,134,139,146,247,250,251,252,253


In [66]:
"""
对测试集进行转化
"""
print(dfTest.columns)

Index(['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtyp

In [67]:
"""
对测试集进行转化
"""
test_ids = dfTest['id'].values.tolist()
dfTest.drop(['id'],axis=1,inplace=True)

test_feature_index = dfTest.copy()
test_feature_value = dfTest.copy()

for col in test_feature_index.columns:
    if col in IGNORE_COLS:
        test_feature_index.drop(col,axis=1,inplace=True)
        test_feature_value.drop(col,axis=1,inplace=True)
        continue
    elif col in NUMERIC_COLS:
        test_feature_index[col] = feature_dict[col]
    else:
        test_feature_index[col] = test_feature_index[col].map(feature_dict[col])
        test_feature_value[col] = 1

In [68]:
import tensorflow as tf
import numpy as np

"""模型参数（这个是需要继续param tuning的）"""
dfm_params = {
    "use_fm":True,
    "use_deep":True,
    "embedding_size":8,
    "dropout_fm":[1.0,1.0],
    "deep_layers":[32,32],
    "dropout_deep":[0.5,0.5,0.5],
    "deep_layer_activation":tf.nn.relu,
    "epoch":30,
    "batch_size":1024,
    "learning_rate":0.001,
    "optimizer":"adam",
    "batch_norm":1,
    "batch_norm_decay":0.995,
    "l2_reg":0.01,
    "verbose":True,
    "eval_metric":'gini_norm',
    "random_seed":3
}
dfm_params['feature_size'] = total_feature
dfm_params['field_size'] = len(train_feature_index.columns)

In [69]:
dfm_params

{'batch_norm': 1,
 'batch_norm_decay': 0.995,
 'batch_size': 1024,
 'deep_layer_activation': <function tensorflow.python.ops.gen_nn_ops.relu>,
 'deep_layers': [32, 32],
 'dropout_deep': [0.5, 0.5, 0.5],
 'dropout_fm': [1.0, 1.0],
 'embedding_size': 8,
 'epoch': 30,
 'eval_metric': 'gini_norm',
 'feature_size': 254,
 'field_size': 37,
 'l2_reg': 0.01,
 'learning_rate': 0.001,
 'optimizer': 'adam',
 'random_seed': 3,
 'use_deep': True,
 'use_fm': True,
 'verbose': True}

[DeepFM架构介绍]()

DeepFM从架构上可以拆解成FM部分(其实是FFM)和DNN部分。

![](https://imgconvert.csdnimg.cn/aHR0cHM6Ly9waWMyLnpoaW1nLmNvbS84MC92Mi1lZmQ1ZWEyNGQ2NWJhNjkyYjc1Mjk5N2M1ZjUwNjE4Ml9oZC5wbmc)

最底层是输入的稀疏原始特征(One-hot结果)，将特征按照Field进行区分，不理解Field含义的同学请学习下FFM。接着进入了Embedding层，将特征限定到有限的向量空间。Embedding层启到一个压缩的作用，不然推荐场景数据是非常稀疏的，比如一个用户A到电商平台B买东西，A可能只购买了一个商品，但是电商平台B有1000个商品，那么购买产品这个特征通过one-hot衍生就是一个1000维度向量，而且在1000纬度里只有1维度有值。Embedding层会缓解参数爆炸。

接着进入FM层和DNN层，两者共用底层的embedding数据，所以效率很高，整个数据embedding只需要计算一次。

在**FM层**，主要的工作是通过特征间交叉得到低阶特征，**以二阶特征为主**。这里面指的注意的是，**FM算法的特征交叉不是直接拿原始特征相互交叉计算，而是交叉特征因子分解后的结果**，这样更能挖掘出特征深层的信息。

![](https://imgconvert.csdnimg.cn/aHR0cHM6Ly9waWMzLnpoaW1nLmNvbS84MC92Mi1lN2IzMzI2MjNkY2Q0ZDIwNGE4OGY1YWNiMWFiNTFhMV9oZC5wbmc)

而在**DNN层**就没有特别多要说的点，就是利用深度学习数据的在网络间的传播性得到高阶特征。

![](https://imgconvert.csdnimg.cn/aHR0cHM6Ly9waWMyLnpoaW1nLmNvbS84MC92Mi0wNzNjYTdlYmMwZjNlYmI3YWU2NjM2OGIxN2Q4MGUyMF9oZC5wbmc)

那最终二分类结果怎么判定也是比较粗暴，利用一个sigmoid求和函数的方式，分别参考FM和DNN的结果做出判断：

![](https://imgconvert.csdnimg.cn/aHR0cHM6Ly9waWMyLnpoaW1nLmNvbS84MC92Mi0yYjczNjQzNDI3YzcxOGJiMjc5MDk4MzM0YzM3NjM0NV9oZC5wbmc)

DeepFM模型代码来源：https://github.com/ChenglongChen/tensorflow-DeepFM/blob/master/example/main.py

In [70]:
"""开始建立模型"""
#feat_index是特征的一个序号，主要用于通过embedding_lookup选择我们的embedding。
#feat_value是对应的特征值，如果是离散特征的话，就是1，如果不是离散特征的话，就保留原来的特征值。label是实际值。还定义了两个dropout来防止过拟合。

feat_index = tf.placeholder(tf.int32,shape=[None,None],name='feat_index')
feat_value = tf.placeholder(tf.float32,shape=[None,None],name='feat_value')

label = tf.placeholder(tf.float32,shape=[None,1],name='label')

"""建立weights"""
weights = dict()

In [71]:
#embeddings
weights['feature_embeddings'] = tf.Variable(
    tf.random_normal([dfm_params['feature_size'],
                      dfm_params['embedding_size']],
                      0.0,
                      0.01
                     ),
    name='feature_embeddings'
    )

weights['feature_bias'] = tf.Variable(tf.random_normal([dfm_params['feature_size'],1],0.0,1.0),
                                      name='feature_bias')

In [72]:
# deep layers
num_layer = len(dfm_params['deep_layers'])
input_size = dfm_params['field_size'] * dfm_params['embedding_size']
glorot = np.sqrt(2.0/(input_size + dfm_params['deep_layers'][0]))

weights['layer_0'] = tf.Variable(
    np.random.normal(loc=0,scale=glorot,size=(input_size,dfm_params['deep_layers'][0])),dtype=np.float32
)
weights['bias_0'] = tf.Variable(
    np.random.normal(loc=0,scale=glorot,size=(1,dfm_params['deep_layers'][0])),dtype=np.float32
)

for i in range(1,num_layer):
    glorot = np.sqrt(2.0 / (dfm_params['deep_layers'][i - 1] + dfm_params['deep_layers'][i]))
    weights["layer_%d" % i] = tf.Variable(
        np.random.normal(loc=0, scale=glorot, size=(dfm_params['deep_layers'][i - 1], dfm_params['deep_layers'][i])),
        dtype=np.float32)  # layers[i-1] * layers[i]
    weights["bias_%d" % i] = tf.Variable(
        np.random.normal(loc=0, scale=glorot, size=(1, dfm_params['deep_layers'][i])),
        dtype=np.float32)  # 1 * layer[i]

In [73]:
# final concat projection layer
if dfm_params['use_fm'] and dfm_params['use_deep']:
    input_size = dfm_params['field_size'] + dfm_params['embedding_size'] + dfm_params['deep_layers'][-1]
elif dfm_params['use_fm']:
    input_size = dfm_params['field_size'] + dfm_params['embedding_size']
elif dfm_params['use_deep']:
    input_size = dfm_params['deep_layers'][-1]

glorot = np.sqrt(2.0/(input_size + 1))   # neural network glorot initialization
weights['concat_projection'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(input_size,1)),dtype=np.float32)
weights['concat_bias'] = tf.Variable(tf.constant(0.01),dtype=np.float32)

In [74]:
"""Dense embedding layer"""
embeddings = tf.nn.embedding_lookup(weights['feature_embeddings'],feat_index)
reshaped_feat_value = tf.reshape(feat_value,shape=[-1,dfm_params['field_size'],1])

# 这个由sparse features经过dense embedding layer提炼出来这个embedding 将作为FM和Deep layer共同的输入
embeddings = tf.multiply(embeddings,reshaped_feat_value)

In [75]:
embeddings.shape   

TensorShape([Dimension(None), Dimension(37), Dimension(8)])

In [76]:
train_feature_value.shape #每个case的embedding size为 37 (特征数) * 8 (设定值)

(10000, 37)

In [77]:
train_feature_value.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.7,0.2,0.718070,1,1,1,1,1,1,1,1,1,1,1,1,0.400000,0.883679,0.370810,3.605551
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.8,0.4,0.766078,1,1,1,1,1,1,1,1,1,1,1,1,0.316228,0.618817,0.388716,2.449490
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0,-1.000000,1,1,1,1,1,1,1,1,1,1,1,1,0.316228,0.641586,0.347275,3.316625
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.9,0.2,0.580948,1,1,1,1,1,1,1,1,1,1,1,1,0.374166,0.542949,0.294958,2.000000
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.7,0.6,0.840759,1,1,1,1,1,1,1,1,1,1,1,1,0.316070,0.565832,0.365103,2.000000


In [78]:
"""FM part"""
# FM一阶
fm_first_order = tf.nn.embedding_lookup(weights['feature_bias'],feat_index)
fm_first_order = tf.reduce_sum(tf.multiply(fm_first_order,reshaped_feat_value),2)

# FM二阶
summed_features_emb = tf.reduce_sum(embeddings,1)
summed_features_emb_square = tf.square(summed_features_emb)

squared_features_emb = tf.square(embeddings)
squared_sum_features_emb = tf.reduce_sum(squared_features_emb,1)

fm_second_order = 0.5 * tf.subtract(summed_features_emb_square,squared_sum_features_emb)

In [79]:
# 两个order的结果维度还不同
print(fm_first_order.shape)
print(fm_second_order.shape)

(?, 37)
(?, 8)


In [80]:
"""Deep part"""
y_deep = tf.reshape(embeddings,shape=[-1,dfm_params['field_size'] * dfm_params['embedding_size']])

for i in range(0,len(dfm_params['deep_layers'])):
    y_deep = tf.add(tf.matmul(y_deep, weights["layer_%d" %i]), weights["bias_%d"%i])
    y_deep = tf.nn.relu(y_deep)

In [81]:
y_deep.shape

TensorShape([Dimension(None), Dimension(32)])

In [82]:
"""final layer"""
# 输出层，即将前面的偏差、一阶、二阶、深度值加起来，送入sigmoid函数后，输出一个0~1的最终概率值。
if dfm_params['use_fm'] and dfm_params['use_deep']:
    concat_input = tf.concat([fm_first_order,fm_second_order,y_deep],axis=1)
elif dfm_params['use_fm']:
    concat_input = tf.concat([fm_first_order,fm_second_order],axis=1)
elif dfm_params['use_deep']:
    concat_input = y_deep
    
out = tf.nn.sigmoid(tf.add(tf.matmul(concat_input,weights['concat_projection']),weights['concat_bias']))

In [83]:
concat_input.shape   # 最后一层的input为 77 (= FM 一阶 37 + FM 二阶 8 + 深度层 32) 维embedding

TensorShape([Dimension(None), Dimension(77)])

In [84]:
"""loss and optimizer"""
loss = tf.losses.log_loss(tf.reshape(label,(-1,1)), out)
optimizer = tf.train.AdamOptimizer(learning_rate=dfm_params['learning_rate'], beta1=0.9, beta2=0.999,
                                                        epsilon=1e-8).minimize(loss)

"""train"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20):
        epoch_loss,_ = sess.run([loss,optimizer],
                                feed_dict={feat_index:train_feature_index,
                                          feat_value:train_feature_value,
                                           label:train_y
                                           }
                                )
        print("epoch %s,loss is %s" % (str(i),str(epoch_loss)))

epoch 0,loss is 1.1206856
epoch 1,loss is 1.089387
epoch 2,loss is 1.0595767
epoch 3,loss is 1.0320662
epoch 4,loss is 1.0053567
epoch 5,loss is 0.9794094
epoch 6,loss is 0.9543656
epoch 7,loss is 0.9297282
epoch 8,loss is 0.9051535
epoch 9,loss is 0.88057977
epoch 10,loss is 0.8560652
epoch 11,loss is 0.83167166
epoch 12,loss is 0.8074202
epoch 13,loss is 0.7833337
epoch 14,loss is 0.7594871
epoch 15,loss is 0.7359344
epoch 16,loss is 0.71262753
epoch 17,loss is 0.6893477
epoch 18,loss is 0.6655793
epoch 19,loss is 0.6416276


全部代码：

In [ ]:
"""开始建立模型"""
feat_index = tf.placeholder(tf.int32,shape=[None,None],name='feat_index')
feat_value = tf.placeholder(tf.float32,shape=[None,None],name='feat_value')

label = tf.placeholder(tf.float32,shape=[None,1],name='label')

"""建立weights"""
weights = dict()

#embeddings
weights['feature_embeddings'] = tf.Variable(
    tf.random_normal([dfm_params['feature_size'],
                      dfm_params['embedding_size']],
                      0.0,
                      0.01
                     ),
    name='feature_embeddings'
    )

weights['feature_bias'] = tf.Variable(tf.random_normal([dfm_params['feature_size'],1],0.0,1.0),
                                      name='feature_bias')


#deep layers
num_layer = len(dfm_params['deep_layers'])
input_size = dfm_params['field_size'] * dfm_params['embedding_size']
glorot = np.sqrt(2.0/(input_size + dfm_params['deep_layers'][0]))

weights['layer_0'] = tf.Variable(
    np.random.normal(loc=0,scale=glorot,size=(input_size,dfm_params['deep_layers'][0])),dtype=np.float32
)
weights['bias_0'] = tf.Variable(
    np.random.normal(loc=0,scale=glorot,size=(1,dfm_params['deep_layers'][0])),dtype=np.float32
)


for i in range(1,num_layer):
    glorot = np.sqrt(2.0 / (dfm_params['deep_layers'][i - 1] + dfm_params['deep_layers'][i]))
    weights["layer_%d" % i] = tf.Variable(
        np.random.normal(loc=0, scale=glorot, size=(dfm_params['deep_layers'][i - 1], dfm_params['deep_layers'][i])),
        dtype=np.float32)  # layers[i-1] * layers[i]
    weights["bias_%d" % i] = tf.Variable(
        np.random.normal(loc=0, scale=glorot, size=(1, dfm_params['deep_layers'][i])),
        dtype=np.float32)  # 1 * layer[i]


# final concat projection layer
if dfm_params['use_fm'] and dfm_params['use_deep']:
    input_size = dfm_params['field_size'] + dfm_params['embedding_size'] + dfm_params['deep_layers'][-1]
elif dfm_params['use_fm']:
    input_size = dfm_params['field_size'] + dfm_params['embedding_size']
elif dfm_params['use_deep']:
    input_size = dfm_params['deep_layers'][-1]

glorot = np.sqrt(2.0/(input_size + 1))
weights['concat_projection'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(input_size,1)),dtype=np.float32)
weights['concat_bias'] = tf.Variable(tf.constant(0.01),dtype=np.float32)

"""Dense embedding layer"""
embeddings = tf.nn.embedding_lookup(weights['feature_embeddings'],feat_index)
reshaped_feat_value = tf.reshape(feat_value,shape=[-1,dfm_params['field_size'],1])

embeddings = tf.multiply(embeddings,reshaped_feat_value)


"""FM part"""
# FM一阶
fm_first_order = tf.nn.embedding_lookup(weights['feature_bias'],feat_index)
fm_first_order = tf.reduce_sum(tf.multiply(fm_first_order,reshaped_feat_value),2)


# FM二阶
summed_features_emb = tf.reduce_sum(embeddings,1)
summed_features_emb_square = tf.square(summed_features_emb)

squared_features_emb = tf.square(embeddings)
squared_sum_features_emb = tf.reduce_sum(squared_features_emb,1)

fm_second_order = 0.5 * tf.subtract(summed_features_emb_square,squared_sum_features_emb)


"""Deep part"""
y_deep = tf.reshape(embeddings,shape=[-1,dfm_params['field_size'] * dfm_params['embedding_size']])

for i in range(0,len(dfm_params['deep_layers'])):
    y_deep = tf.add(tf.matmul(y_deep,weights["layer_%d" %i]), weights["bias_%d"%i])
    y_deep = tf.nn.relu(y_deep)
    

"""final layer"""
# 输出层，即将前面的偏差、一阶、二阶、深度值加起来，送入sigmoid函数后，输出一个0~1的最终概率值。
if dfm_params['use_fm'] and dfm_params['use_deep']:
    concat_input = tf.concat([fm_first_order,fm_second_order,y_deep],axis=1)
elif dfm_params['use_fm']:
    concat_input = tf.concat([fm_first_order,fm_second_order],axis=1)
elif dfm_params['use_deep']:
    concat_input = y_deep
    
out = tf.nn.sigmoid(tf.add(tf.matmul(concat_input,weights['concat_projection']),weights['concat_bias']))

"""loss and optimizer"""
loss = tf.losses.log_loss(tf.reshape(label,(-1,1)), out)
optimizer = tf.train.AdamOptimizer(learning_rate=dfm_params['learning_rate'], beta1=0.9, beta2=0.999,
                                                        epsilon=1e-8).minimize(loss)

"""train"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20):
        epoch_loss,_ = sess.run([loss,optimizer],feed_dict={feat_index:train_feature_index,
                             feat_value:train_feature_value,
                             label:train_y})
        print("epoch %s,loss is %s" % (str(i),str(epoch_loss)))

epoch 0,loss is 1.8298997
epoch 1,loss is 1.7893096
epoch 2,loss is 1.7489114
epoch 3,loss is 1.7089047
epoch 4,loss is 1.6697114
epoch 5,loss is 1.6314086
epoch 6,loss is 1.5934154
epoch 7,loss is 1.5555546
epoch 8,loss is 1.5178305
epoch 9,loss is 1.4803187
epoch 10,loss is 1.4428422
epoch 11,loss is 1.4051893
epoch 12,loss is 1.3672795
epoch 13,loss is 1.3290857
epoch 14,loss is 1.2906075
epoch 15,loss is 1.2518283
epoch 16,loss is 1.2128348
epoch 17,loss is 1.173999
epoch 18,loss is 1.1353949
epoch 19,loss is 1.0967705


# Appendix

D2L实现方法：https://d2l.ai/chapter_recommender-systems/deepfm.html